In [0]:
kafka_bootstrap_servers = "pkc-oxqxx9.us-east-1.aws.confluent.cloud:9092"
kafka_api_key = "GDYCUMDX5QRPX2WS"
kafka_api_secret = "cflt9HN3ZSyS33B342mEmdOWwfm6tYShkf6sBnUwi44G6ZjASGQ8eVBfkqeRPyTQ"

orders_topic = "orders_stream"
inventory_topic = "inventory_stream"


# Orders

In [0]:
from pyspark import pipelines as dp
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType

orders_schema = StructType([
    StructField("order_id", StringType()),
    StructField("customer_id", StringType()),
    StructField("product_id", StringType()),
    StructField("store_id", StringType()),
    StructField("order_date", StringType()),
    StructField("quantity", IntegerType()),
    StructField("stock_date", StringType()),
    StructField("payment_type", StringType()),
    StructField("price", DoubleType()),
    StructField("order_ts", TimestampType())
])

@dp.table(
    name="orders_streaming_table",
    comment="Streaming table ingesting orders from Kafka"
)
def orders_streaming_table():
    df_kafka = (
        spark.readStream
        .format("kafka")
        .option("kafka.bootstrap.servers", kafka_bootstrap_servers)
        .option("subscribe", orders_topic)
        .option("kafka.security.protocol", "SASL_SSL")
        .option("kafka.sasl.mechanism", "PLAIN")
        .option(
            "kafka.sasl.jaas.config",
            f"kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{kafka_api_key}' password='{kafka_api_secret}';"
        )
        .option("startingOffsets", "earliest")
        .load()
    )
    return (
        df_kafka
        .selectExpr("CAST(value AS STRING) as value")
        .select(from_json(col("value"), orders_schema).alias("data"))
        .select("data.*")
    )

In [0]:
from pyspark.sql.functions import count

# 1. Create a streaming DataFrame by reading the table as a stream
streaming_df = (spark.readStream
                .table("maven_uc.bronze_dlt.orders_streaming_table"))

# 2. Perform the aggregation
total_orders_df = streaming_df.select(count("order_id").alias("Total_orders"))

# 3. Display the live output in the notebook
display(total_orders_df)

Databricks visualization. Run in Databricks to view.


# Inventory

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType

inventory_schema = StructType([
    StructField("product_id", StringType()),
    StructField("store_id", StringType()),
    StructField("restock_qty", IntegerType()),
    StructField("quantity_remaining", IntegerType()),
    StructField("restock_date", StringType()),
    StructField("event_ts", TimestampType())
])

In [0]:
from pyspark import pipelines as dp
from pyspark.sql.functions import from_json, col

@dp.table(
    name="inventory_streaming_table",
    comment="Streaming table ingesting inventory from Kafka"
)
def inventory_streaming_table():
    df_kafka = (
        spark.readStream
        .format("kafka")
        .option("kafka.bootstrap.servers", kafka_bootstrap_servers)
        .option("subscribe", inventory_topic)
        .option("kafka.security.protocol", "SASL_SSL")
        .option("kafka.sasl.mechanism", "PLAIN")
        .option(
            "kafka.sasl.jaas.config",
            f"kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{kafka_api_key}' password='{kafka_api_secret}';"
        )
        .option("startingOffsets", "earliest")
        .load()
    )
    return (
        df_kafka
        .selectExpr("CAST(value AS STRING) as value")
        .select(from_json(col("value"), inventory_schema).alias("data"))
        .select("data.*")
    )